In [1]:
from app import create_app
from app.instrument import Instrument
from app.analysis import Quote
with create_app().app_context():
    quote = Quote([b.symbol for b in Instrument.find_etfs(100)], 120)
f'from {quote.start.date()} to {quote.end.date()} - {len(quote.data.columns)}'

'from 2019-02-21 to 2019-06-21 - 85'

In [2]:
stat = quote.statistics(21)
stat = stat[(stat['drawdown'] < 8) & (stat['21-mean'] > 0.8)]
stat

,21-len,21-mean,21-std,21-shrp,drawdown
Symbols,,,,,
SPLV,64,1.989199,1.351612,1.348414,1.971361
FINX,64,2.192274,2.360783,0.858024,5.312173
XLU,64,1.516285,1.910099,0.706570,3.426052
XAR,64,2.275932,3.360279,0.627706,6.827853
ICLN,64,1.782633,2.873275,0.562413,4.109596
VIG,64,1.357317,2.348643,0.506952,4.634193
VOOG,64,1.309920,2.788859,0.409936,6.391444
SPYG,64,1.299128,2.770475,0.408761,6.359873
DGRO,64,0.925188,2.483952,0.305369,5.769734


In [7]:
quote.setup_mask(stat.index)
quote.least_correlated_portfolio(21, 2, ['SPLV'], cr=3, dr=3)

['SPLV', 'FINX'] 3.788149944225284 0.54148906839553 1.456706758047411 -2.206437535103539
['SPLV', 'XLU'] 3.367608777719975 0.7280483184802429 1.0794825893801903 -2.054983945861325


['SPLV', 'XLU']

In [11]:
quote.setup_mask(['SPLV', 'XLU'])
quote.find_optimal_ratio(21, 1)

     fun: 1.8252326449006429
     jac: array([3.65044618, 3.64982736])
 message: 'Optimization terminated successfully.'
    nfev: 16
     nit: 8
    njev: 4
  status: 0
 success: True
       x: array([ 1.03092219, -0.03092219])


({'SPLV': 1.030922192223077, 'XLU': -0.030922192223076916}, 2.0038, 1.351)